In [7]:
import os
from dotenv import load_dotenv
import requests
import openai

In [8]:
load_dotenv()

True

In [9]:


FINNHUB_API_KEY = os.getenv("FINNHUB")
BASE_URL = "https://finnhub.io/api/v1/"

headers = {
    "X-Finnhub-Token": FINNHUB_API_KEY
}

def get_ratios_for_ticker(ticker):
    """Fetch P/E, P/B, and P/S ratios for the given ticker."""
    response = requests.get(f"{BASE_URL}stock/metric?symbol={ticker}&metric=all", headers=headers)
    data = response.json()
    
    pe_ratio = data['metric']['peNormalizedAnnual']
    pb_ratio = data['metric']['pbAnnual']
    ps_ratio = data['metric']['psAnnual']
    
    return pe_ratio, pb_ratio, ps_ratio

def get_peers_of_ticker(ticker):
    """Fetch the list of peers for the given ticker."""
    response = requests.get(f"{BASE_URL}stock/peers?symbol={ticker}", headers=headers)
    peers = response.json()
    return peers

def get_peer_ratios(ticker):
    """Fetch average P/E, P/B, and P/S ratios for the peers of the given ticker."""
    peers = get_peers_of_ticker(ticker)
    pe_ratios, pb_ratios, ps_ratios = [], [], []

    for peer in peers:
        try:
            pe, pb, ps = get_ratios_for_ticker(peer)
            if pe is not None:
                pe_ratios.append(pe)
            if pb is not None:
                pb_ratios.append(pb)
            if ps is not None:
                ps_ratios.append(ps)
        except:
            pass

    avg_pe = sum(pe_ratios) / len(pe_ratios) if pe_ratios else None
    avg_pb = sum(pb_ratios) / len(pb_ratios) if pb_ratios else None
    avg_ps = sum(ps_ratios) / len(ps_ratios) if ps_ratios else None

    return avg_pe, avg_pb, avg_ps

def main():
    ticker = input("Enter the ticker symbol: ").upper()
    
    # Fetch ratios for the given ticker
    pe, pb, ps = get_ratios_for_ticker(ticker)
    print(f"\n{ticker} Ratios:")
    print(f"P/E Ratio: {pe}")
    print(f"P/B Ratio: {pb}")
    print(f"P/S Ratio: {ps}")

    # Fetch average ratios for the peers of the given ticker
    avg_pe, avg_pb, avg_ps = get_peer_ratios(ticker)
    print("\nAverage Ratios Among Peers:")
    print(f"P/E Ratio: {avg_pe}")
    print(f"P/B Ratio: {avg_pb}")
    print(f"P/S Ratio: {avg_ps}")

if __name__ == "__main__":
    main()


Enter the ticker symbol:  JPM



JPM Ratios:
P/E Ratio: 10.8581
P/B Ratio: 1.3455
P/S Ratio: 3.22798

Average Ratios Among Peers:
P/E Ratio: 7.6446125
P/B Ratio: 1.100275
P/S Ratio: 1.961105


In [10]:

# Initialize the OpenAI API with your key
openai.api_key = os.getenv("AI_KEY")  # Replace with your actual key

def analyze_ratios_with_openai(ticker, pe, pb, ps, avg_pe, avg_pb, avg_ps):
    # Constructing a prompt for the OpenAI API
    prompt = f"""
    Analyze the financial ratios for the company with ticker symbol {ticker}:

    - P/E Ratio (Price-to-Earnings Ratio) for {ticker}: {pe}
      Average P/E Ratio among peers: {avg_pe}
      
    - P/B Ratio (Price-to-Book Ratio) for {ticker}: {pb}
      Average P/B Ratio among peers: {avg_pb}

    - P/S Ratio (Price-to-Sales Ratio) for {ticker}: {ps}
      Average P/S Ratio among peers: {avg_ps}

    Based on this data, provide an analysis of the company's financial standing compared to its peers.
    """

    # Querying the OpenAI API
    response = openai.Completion.create(engine="text-davinci-002", prompt=prompt, max_tokens=200)
    analysis = response.choices[0].text.strip()

    return analysis

# Example usage:
ticker = "JPM"
pe, pb, ps = get_ratios_for_ticker(ticker)
avg_pe, avg_pb, avg_ps = get_peer_ratios(ticker)

analysis_result = analyze_ratios_with_openai(ticker, pe, pb, ps, avg_pe, avg_pb, avg_ps)
print(analysis_result)

Market Valuation Ratios
1. Price / Earnings or P/E Ratio (10.8581)
2. Price / Earnings Growth or PEG Ratio
3. Price / Book Ratio or P/B Ratio (1.3455)
4. Price / Sales Ratio or P/S Ratio (3.22798)
5. Dividend Yield
 
 
Based on the analysis, JPMorgan's P/E, P/B and P/S ratios are all significantly higher than their peers. This indicates that the market is valuing the company at a higher price in comparison to its earnings, book value and sales. In general, a higher P/E ratio means that investors are willing to pay a higher price for a stock because they believe that the company will be more profitable in the future. The higher ratios could also indicate that the market is expecting the company's earnings to grow at a faster rate than its peers in the future
